In [1]:
import torch 
from torch import nn
from torch.nn import functional as F

# 1.A Custiom Block

In [2]:
net =nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
net(X)

tensor([[ 0.2482,  0.1047,  0.1085, -0.2814, -0.0181, -0.0281,  0.0921,  0.2084,
          0.0006, -0.0440],
        [ 0.2679, -0.0356,  0.0075, -0.2893, -0.0146, -0.0293,  0.0434,  0.1277,
          0.0724, -0.0082]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out=nn.Linear(256,10)
        
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [6]:
net=MLP()
net(X)

tensor([[-0.0995,  0.2641,  0.0883,  0.1314, -0.1104, -0.0668, -0.0037,  0.0107,
         -0.1769, -0.0578],
        [-0.0911,  0.2748,  0.0806,  0.2331, -0.0864, -0.0078,  0.0277,  0.0858,
         -0.0920, -0.0419]], grad_fn=<AddmmBackward0>)

# 2.The Sequential Block

In [14]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)]=module
            
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X

In [15]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1852, -0.1021,  0.0305, -0.0819, -0.0244, -0.0375, -0.0892, -0.1231,
          0.0833,  0.0820],
        [ 0.2589, -0.0257,  0.1442, -0.1545,  0.0563, -0.0340, -0.1775, -0.1355,
         -0.0693,  0.1273]], grad_fn=<AddmmBackward0>)

In [18]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)
        
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        X=self.linear(X)
        while X.abs().sum()>1:
            X /=2
        return X.sum()

In [19]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0336, grad_fn=<SumBackward0>)

In [20]:
class NestMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self,X):
        return self.linear(self.net(X))

In [21]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1952, grad_fn=<SumBackward0>)